# 🗄️ PySpark + MySQL - Configuración y Uso

Notebook especializado para trabajar con PySpark y MySQL sin problemas de configuración.

## 🎯 Objetivos:
- Configurar PySpark para MySQL automáticamente
- Probar conectividad
- Cargar y procesar datos
- Optimizar performance
- Mejores prácticas

## ⚙️ 1. Configuración Inicial

In [ ]:
# Importar librerías necesarias
import sys
sys.path.append('../src')

from etl.mysql_spark import create_mysql_spark_connector
from pyspark.sql import functions as F
from pyspark.sql.types import *
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("📚 Librerías importadas correctamente")

## 🔗 2. Crear Conexión MySQL + Spark

In [ ]:
# Crear conector (ajustar config.yaml con tus credenciales)
connector = create_mysql_spark_connector('../config.yaml')

# Verificar que Spark esté funcionando
spark = connector.spark
print(f"✅ Spark versión: {spark.version}")
print(f"✅ Aplicación: {spark.sparkContext.appName}")
print(f"✅ Master: {spark.sparkContext.master}")
print(f"✅ Spark UI: {spark.sparkContext.uiWebUrl}")

## 🧪 3. Probar Conectividad MySQL

In [ ]:
# Probar conexión directa a MySQL
if connector.test_mysql_connection():
    print("🎉 ¡Conexión a MySQL exitosa!")
else:
    print("❌ Problema de conexión a MySQL")
    print("\n🔧 Verifica en config.yaml:")
    print("   - host, port, database")
    print("   - user, password")
    print("   - Que el servidor MySQL esté ejecutándose")

## 📋 4. Explorar Base de Datos

In [ ]:
# Listar todas las tablas disponibles
tables = connector.list_tables()

print(f"📊 Tablas encontradas ({len(tables)}):")
for i, table in enumerate(tables, 1):
    print(f"  {i}. {table}")

if not tables:
    print("⚠️  No se encontraron tablas o no hay permisos")

In [ ]:
# Si tienes tablas, obtener información de una
if tables:
    # Cambiar por el nombre de tu tabla
    table_name = tables[0]  # Usar la primera tabla encontrada
    
    print(f"🔍 Analizando tabla: {table_name}")
    table_info = connector.get_table_info(table_name)
    
    if table_info:
        print(f"   📏 Total filas: {table_info['total_rows']:,}")
        print(f"   📋 Columnas ({len(table_info['columns'])}): {', '.join(table_info['columns'])}")
        print(f"   🎯 Tipos de datos:")
        for col, dtype in table_info['dtypes']:
            print(f"      {col}: {dtype}")

## 📊 5. Cargar Datos con Spark

In [ ]:
# Método 1: Cargar tabla completa (para tablas pequeñas-medianas)
if tables:
    table_name = tables[0]  # Ajustar según tu tabla
    
    # Cargar muestra primero para tablas grandes
    print(f"📥 Cargando muestra de {table_name}...")
    df_sample = connector.read_table(table_name, sample_fraction=0.1)  # 10% de muestra
    
    print(f"✅ Muestra cargada: {df_sample.count():,} filas")
    print("\n🔍 Primeras 5 filas:")
    df_sample.show(5)
    
    print("\n📊 Schema:")
    df_sample.printSchema()

In [ ]:
# Método 2: Query personalizada
if tables:
    table_name = tables[0]
    
    # Ejemplo de query personalizada
    custom_query = f"""
    SELECT * 
    FROM {table_name} 
    LIMIT 1000
    """
    
    print("📝 Ejecutando query personalizada...")
    df_query = connector.read_query(custom_query)
    
    print(f"✅ Query ejecutada: {df_query.count():,} filas")
    df_query.show(3)

In [ ]:
# Método 3: Carga particionada (para tablas muy grandes)
# Requiere una columna numérica para particionar

# Ejemplo (ajustar según tu tabla):
# df_partitioned = connector.read_partitioned(
#     table_name="mi_tabla",
#     partition_column="id",  # Columna numérica
#     lower_bound=1,
#     upper_bound=100000,
#     num_partitions=8
# )

print("💡 Método 3 comentado - descomentar y ajustar según tu tabla")

## 🔄 6. Procesamiento de Datos

In [ ]:
# Trabajar con el DataFrame cargado
if 'df_sample' in locals():
    df = df_sample  # Usar la muestra cargada
    
    print("🔄 Realizando transformaciones...")
    
    # Optimizar DataFrame
    df = connector.optimize_dataframe(df, cache=True)
    
    # Análisis básico
    print(f"📊 Particiones: {df.rdd.getNumPartitions()}")
    print(f"📏 Total filas: {df.count():,}")
    
    # Estadísticas descriptivas
    print("\n📈 Estadísticas:")
    df.describe().show()
    
    # Contar valores nulos
    print("\n🔍 Valores nulos por columna:")
    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
    null_counts.show()

In [ ]:
# Ejemplo de transformaciones comunes
if 'df' in locals():
    print("🛠️ Aplicando transformaciones...")
    
    # Seleccionar columnas específicas (ajustar según tu tabla)
    # df_selected = df.select("col1", "col2", "col3")
    
    # Filtrar datos
    # df_filtered = df.filter(F.col("columna") > 100)
    
    # Agregar nueva columna
    # df_with_new_col = df.withColumn("nueva_col", F.lit("valor"))
    
    # Agrupaciones
    # df_grouped = df.groupBy("categoria").agg(
    #     F.count("*").alias("total"),
    #     F.avg("valor").alias("promedio")
    # )
    
    print("✅ Transformaciones definidas (descomentar según necesites)")

## 💾 7. Guardar Resultados

In [ ]:
# Guardar en formato Parquet (recomendado)
if 'df' in locals():
    output_path = "../data/processed/mysql_spark_output"
    
    print(f"💾 Guardando datos en: {output_path}")
    
    # Guardar como Parquet
    df.coalesce(1).write.mode("overwrite").parquet(output_path + "/parquet")
    
    # Guardar como CSV
    df.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_path + "/csv")
    
    print("✅ Datos guardados exitosamente")

In [ ]:
# Opcional: Escribir de vuelta a MySQL
# Crear tabla nueva con los datos procesados

# if 'df' in locals():
#     print("🔄 Escribiendo datos procesados de vuelta a MySQL...")
#     
#     connector.write_table(
#         df=df_sample.limit(100),  # Solo primeras 100 filas como ejemplo
#         table_name="tabla_procesada_spark",
#         mode="overwrite",
#         batch_size=500
#     )
#     
#     print("✅ Datos escritos a MySQL")

print("💡 Escritura a MySQL comentada - descomentar si necesitas")

## 📊 8. Monitoreo y Performance

In [ ]:
# Información del contexto Spark
sc = spark.sparkContext

print("📊 INFORMACIÓN DE SPARK")
print(f"   🎯 Aplicación ID: {sc.applicationId}")
print(f"   🌐 Spark UI: {sc.uiWebUrl}")
print(f"   💻 Master: {sc.master}")
print(f"   🔧 Configuraciones activas:")

# Mostrar configuraciones importantes
important_configs = [
    'spark.driver.memory',
    'spark.executor.memory', 
    'spark.sql.adaptive.enabled',
    'spark.jars.packages'
]

for config in important_configs:
    value = spark.conf.get(config, 'No configurado')
    print(f"      {config}: {value}")

In [ ]:
# Verificar drivers JDBC disponibles
print("🔌 DRIVERS JDBC DISPONIBLES")
try:
    # Verificar que el driver MySQL esté cargado
    test_df = spark.sql("SELECT 1 as test")
    print("   ✅ Spark SQL funcionando")
    
    # Mostrar packages cargados
    packages = spark.conf.get('spark.jars.packages', 'No configurado')
    print(f"   📦 Packages: {packages}")
    
except Exception as e:
    print(f"   ❌ Error: {e}")

## 🧹 9. Limpieza

In [ ]:
# Limpiar cache si fue usado
if 'df' in locals():
    df.unpersist()
    print("🧹 Cache limpiado")

# Opcional: Cerrar Spark (descomentar si quieres)
# connector.close()
# print("🔚 SparkSession cerrada")

print("✅ Limpieza completada")

## 🛠️ 10. Troubleshooting y Tips

### ❌ Problemas Comunes:

#### 1. **Error de conexión MySQL**
```
Communications link failure
```
**Solución:**
- Verificar que MySQL esté ejecutándose
- Revisar host, port, user, password en config.yaml
- Verificar permisos del usuario MySQL

#### 2. **Driver JDBC no encontrado**
```
No suitable driver found
```
**Solución:**
- El conector descarga automáticamente el driver
- Verificar conexión a internet
- Revisar spark.jars.packages en configuración

#### 3. **OutOfMemoryError**
```
Java heap space
```
**Solución:**
- Aumentar spark.driver.memory en config.yaml
- Usar muestras más pequeñas
- Particionar datos grandes

### ✅ Mejores Prácticas:

1. **Performance:**
   - Usar `cache()` para DataFrames reutilizados
   - Particionar tablas grandes
   - Filtrar en la query MySQL cuando sea posible

2. **Memoria:**
   - Empezar con muestras pequeñas
   - Aumentar memoria gradualmente
   - Usar `coalesce()` para reducir particiones

3. **Desarrollo:**
   - Probar conexiones antes de cargas grandes
   - Usar LIMIT en queries de prueba
   - Monitorear Spark UI durante desarrollo

### 🔧 Configuración Recomendada:

Para tablas **pequeñas** (< 1M filas):
```yaml
spark:
  driver_memory: "2g"
  executor_memory: "2g"
```

Para tablas **medianas** (1M - 10M filas):
```yaml
spark:
  driver_memory: "4g"
  executor_memory: "4g"
```

Para tablas **grandes** (> 10M filas):
```yaml
spark:
  driver_memory: "8g"
  executor_memory: "8g"
```